## Test case set up 

In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from collections import Counter

x_dim = 10
y_dim = 20
m1 = 20
m2 = 20
scenario_num = 2
p1 = 500
p2 = 500
p3 = 500

x_fix = np.random.choice([0,1], size = x_dim, p = [1./5, 4./5])
y_fix_list = []
for w in range(scenario_num):
    if w % 3 == 0:
        y_fix_list.append(np.random.choice([0,1], size = y_dim, p = [1./5, 4./5]))
    if w % 3 == 1:
        y_fix_list.append(np.random.choice([0,1], size = y_dim, p = [1./5, 4./5]))
    if w % 3 == 2:
        y_fix_list.append(np.random.choice([0,1], size = y_dim, p = [1./5, 4./5]))

c = np.ones(x_dim)
Q = np.diag(np.ones(x_dim))
A = np.random.choice([0,1], size = (m1,x_dim), p = [4./5, 1./5])
b = A @ x_fix
q_list = []
D_list = []
T_list = []
W_list = []
h_list = []
omega_list = [] # price 
p_xi_list = np.ones(scenario_num)/scenario_num
for w in range(scenario_num):
    q_list.append(np.ones(y_dim)*(w+1))
    D_list.append(np.diag(np.ones(y_dim))*(w+1))
    T_list.append(np.random.choice([0,1], size = (m2,x_dim), p = [4./5, 1./5]))
    W_list.append(np.ones((m2,y_dim))*(w+1))
    omega_list.append(np.zeros(x_dim)) # initiate price to zero
for w in range(scenario_num):
    h_list.append(T_list[w]@x_fix + W_list[w]@y_fix_list[w])

## Gurobi to solve the original large two-stage quadratic problem (w/o PHA or QUBO form)

In [ ]:
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()

problem = gp.Model(env=env)
problem.setParam('OutputFlag', False)

# Gurobi: Initialize variables
x = problem.addMVar(x_dim, vtype=GRB.BINARY, name = 'x')
y = []
for w in range(scenario_num):
    y_w = problem.addMVar(y_dim, vtype=GRB.BINARY, name = 'y%d' %w)
    y.append(y_w)

# Gurobi: Add constraints
problem.addConstr(A@x == b)
for w in range(scenario_num):
    problem.addConstr(T_list[w]@x + W_list[w]@y[w] == h_list[w])

# Gurobi: Set up objective
obj = x@np.diag(c)@x + x@(0.5*Q)@x 
for w in range(scenario_num): 
    obj += y[w]@(p_xi_list[w]*np.diag(q_list[w]))@y[w] + y[w]@(p_xi_list[w]*0.5*D_list[w])@y[w]
    
problem.setObjective(obj, GRB.MINIMIZE)
problem.params.NonConvex = 2
problem.optimize()
x = []
y = []
for v in problem.getVars():
    if 'x' in v.varName:
        x.append(v.x)
    if 'y' in v.varName:
        y.append(v.x)

print('x solution: ' )
print(x)
print('y solution: ' )
print(y)
print('Running time: ')
print(problem.Runtime)

## PHA with Gurobi (w/o QUBO form)

In [ ]:
env = gp.Env(empty=True)
env.setParam("LogToConsole", 0)
env.setParam("LogFile", 'quadratic_two_stage_pha.txt')
env.start()

num_iterations = 100
x_list = [0]*num_iterations
y_list = [0]*num_iterations
all_run_time = []
cur_iter = 0


x_all_scenarios = []
y_all_scenarios = []
run_time = []
objectives = []

# PHA: Initialization 
for w in range(scenario_num):
    problem = gp.Model(env=env)
    
    # Gurobi: Initialize variables
    x = problem.addMVar(x_dim, vtype=GRB.BINARY, name = 'x')
    y = problem.addMVar(y_dim, vtype=GRB.BINARY, name = 'y')
    
    # Gurobi: Add constraint 
    problem.addConstr(A@x == b)
    problem.addConstr(T_list[w]@x + W_list[w]@y == h_list[w])
    
    # Gurobi: Set objective
    obj = x@np.diag(c)@x + x@(0.5*Q)@x + y@np.diag(q_list[w])@y + y@(0.5*D_list[w])@y 
    problem.setObjective(obj, GRB.MINIMIZE)
    problem.params.NonConvex = 2
    problem.optimize()
    objectives.append(problem.getObjective().getValue())
    x_w = []
    y_w = []
    for v in problem.getVars():
        if 'x' in v.varName:
            x_w.append(v.x)
        if 'y' in v.varName:
            y_w.append(v.x)
    x_all_scenarios.append(x_w)
    y_all_scenarios.append(y_w)
    run_time.append(problem.Runtime)
    all_run_time.append(problem.Runtime)

x_list[cur_iter] = x_all_scenarios
y_list[cur_iter]= y_all_scenarios
print('----------Iteration %d----------' %cur_iter)
print('x solution: ' )
print(x_all_scenarios)
print('y solution: ' )
print(y_all_scenarios)
print('Running time: ')
print(run_time)
for w in range(scenario_num):
    print('Ojective for scenario %d' %w)
    print(objectives[w])
cur_iter += 1

omega_list_copy = []
for w in range(scenario_num):
    omega_list_copy.append(np.zeros(x_dim)) # initiate price to zero

while (not x_all_scenarios.count(x_all_scenarios[0]) == len(x_all_scenarios)) and (cur_iter < num_iterations):
    # PHA: Aggregation
    x_hat = np.zeros(x_dim)
    for w in range(scenario_num):
        x_hat += p_xi_list[w]*np.asarray(x_all_scenarios[w])
            
    # PHA: Decomposition
    x_all_scenarios = []
    y_all_scenarios = []
    run_time = []
    objectives = []
    for w in range(scenario_num):
        problem = gp.Model(env=env)
        
        # Gurobi: Initialize variables
        x = problem.addMVar(x_dim, vtype=GRB.BINARY, name = 'x')
        y = problem.addMVar(y_dim, vtype=GRB.BINARY, name = 'y')
        
        # Gurobi: Add constraints
        problem.addConstr(A@x == b)
        problem.addConstr(T_list[w]@x + W_list[w]@y[w] == h_list[w])
        
        # Gurobi: Set objective
        obj = x@np.diag(c)@x + x@(0.5*Q)@x + y@np.diag(q_list[w])@y + y@(0.5*D_list[w])@y  
        obj += omega_list_copy[w]@x + (x - x_hat)@(0.5*p3*np.identity(x_dim))@(x - x_hat)
        problem.setObjective(obj, GRB.MINIMIZE)
        problem.params.NonConvex = 2
        problem.optimize()
        objectives.append(problem.getObjective().getValue())
        x_w = []
        y_w = []
        for v in problem.getVars():
            if 'x' in v.varName:
                x_w.append(v.x)
            if 'y' in v.varName:
                y_w.append(v.x)
        x_all_scenarios.append(x_w)
        y_all_scenarios.append(y_w)
        run_time.append(problem.Runtime)
        all_run_time.append(problem.Runtime)
    
    x_list[cur_iter] = x_all_scenarios
    y_list[cur_iter] = y_all_scenarios
    print('----------Iteration %d----------' %cur_iter)
    print('x solution: ' )
    print(x_all_scenarios)
    print('y solution: ' )
    print(y_all_scenarios)
    print('Running time: ')
    print(run_time)
        
    for w in range(scenario_num):
        print('Ojective for scenario %d' %w)
        print(objectives[w])
    
    # PHA: Price update
    for w in range(scenario_num):
        omega_list_copy[w] += p3*(np.asarray(x_all_scenarios[w]) - x_hat)
    # PHA: Iteration update
    cur_iter += 1

## PHA with Gurobi (w QUBO form) 

In [ ]:
env = gp.Env(empty=True)
env.setParam("LogToConsole", 0)
env.setParam("LogFile", 'quadratic_two_stage_pha.txt')
env.start()

num_iterations = 100
x_list = [0]*num_iterations
y_list = [0]*num_iterations
z_list = [0]*num_iterations
all_run_time = []
cur_iter = 0

x_all_scenarios = []
y_all_scenarios = []
z_all_scenarios = []
run_time = []
objectives = []

# PHA: Initialization 
for w in range(scenario_num):
    problem = gp.Model(env=env)
    
    ##### Compute QUBO Q matrix ######
    dwave_Q = np.zeros((x_dim + y_dim, x_dim + y_dim))
    
    dwave_Q[0:x_dim, 0:x_dim] = np.diag(c) + 0.5*Q + p1*np.matmul(np.transpose(A),A) - 2*p1*np.diag(np.matmul(np.transpose(A),b)) \
                            + p2*np.matmul(np.transpose(T_list[w]),T_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(T_list[w]),h_list[w]))
    dwave_Q[x_dim:(x_dim+y_dim), x_dim:(x_dim+y_dim)] = np.diag(q_list[w]) + 0.5*D_list[w] + \
                            + p2*np.matmul(np.transpose(W_list[w]),W_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(W_list[w]),h_list[w]))
    T_m_W = np.matmul(np.transpose(T_list[w]),W_list[w])
    for i in range(x_dim):
        for j in range(y_dim):
            x_index = i
            y_index = j + x_dim
            dwave_Q[x_index, y_index] = p2*T_m_W[i,j]
            dwave_Q[y_index, x_index] = p2*T_m_W[i,j]
    # np.savetxt("Q" + str(cur_iter) + str(w) + ".csv", dwave_Q, delimiter=",")
    ###################################
    
    z = problem.addMVar(x_dim + y_dim, vtype=GRB.BINARY, name = 'z')
    obj = z@dwave_Q@z + p1*np.matmul(np.transpose(b),b) + p2*np.matmul(np.transpose(h_list[w]),h_list[w])
    
    problem.setObjective(obj, GRB.MINIMIZE)
    problem.optimize()
    objectives.append(problem.getObjective().getValue())
    x_w = []
    y_w = []
    z_w = []
    for v in problem.getVars():
        if 'z' in v.varName:
            z_w.append(v.x)
    x_w = z_w[0:x_dim]
    y_w = z_w[x_dim:(x_dim+y_dim)]
    x_all_scenarios.append(x_w)
    y_all_scenarios.append(y_w)
    z_all_scenarios.append(z_w)
    run_time.append(problem.Runtime)
    all_run_time.append(problem.Runtime)
    
x_list[cur_iter] = x_all_scenarios
y_list[cur_iter]= y_all_scenarios
z_list[cur_iter] = z_all_scenarios
print('----------Iteration %d----------' %cur_iter)
print('x solution: ' )
print(x_all_scenarios)
print('y solution: ' )
print(y_all_scenarios)
print('Running time: ')
print(run_time)
for w in range(scenario_num):
    z_w = z_all_scenarios[w]
    print('Ojective for scenario %d' %w)
    print(objectives[w])

omega_list_copy = []
for w in range(scenario_num):
    omega_list_copy.append(np.zeros(x_dim)) # initiate price to zero

while (not x_all_scenarios.count(x_all_scenarios[0]) == len(x_all_scenarios)) and (cur_iter < num_iterations):
    # PHA: Aggregation
    x_hat = np.zeros(x_dim)
    for w in range(scenario_num):
        x_hat += p_xi_list[w]*np.asarray(x_all_scenarios[w])
            
    # PHA: Decomposition
    x_all_scenarios = []
    y_all_scenarios = []
    z_all_scenarios = []
    run_time = []
    objectives = []
    for w in range(scenario_num):
        problem = gp.Model(env=env)
        
        ##### Compute QUBO Q matrix ######
        dwave_Q = np.zeros((x_dim + y_dim, x_dim + y_dim))
    
        dwave_Q[0:x_dim, 0:x_dim] = np.diag(c) + 0.5*Q + p1*np.matmul(np.transpose(A),A) - 2*p1*np.diag(np.matmul(np.transpose(A),b)) \
                            + p2*np.matmul(np.transpose(T_list[w]),T_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(T_list[w]),h_list[w])) \
                            + np.diag(omega_list_copy[w]) + 0.5*p3*np.identity(x_dim) - p3*np.diag(x_hat)
        dwave_Q[x_dim:(x_dim+y_dim), x_dim:(x_dim+y_dim)] = np.diag(q_list[w]) + 0.5*D_list[w] + \
                            + p2*np.matmul(np.transpose(W_list[w]),W_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(W_list[w]),h_list[w]))
        T_m_W = np.matmul(np.transpose(T_list[w]),W_list[w])
        for i in range(x_dim):
            for j in range(y_dim):
                x_index = i
                y_index = j + x_dim
                dwave_Q[x_index, y_index] = p2*T_m_W[i,j]
                dwave_Q[y_index, x_index] = p2*T_m_W[i,j]
        # np.savetxt("Q" + str(cur_iter) + str(w) + ".csv", dwave_Q, delimiter=",")
        ###################################
    
        z = problem.addMVar(x_dim + y_dim, vtype=GRB.BINARY, name = 'z')
        obj = z@dwave_Q@z + p1*np.matmul(np.transpose(b),b) + p2*np.matmul(np.transpose(h_list[w]),h_list[w]) + 0.5*p3*np.matmul(np.transpose(x_hat),x_hat)
        
        problem.setObjective(obj, GRB.MINIMIZE)
        problem.params.NonConvex = 2
        problem.optimize()
        objectives.append(problem.getObjective().getValue())
        x_w = []
        y_w = []
        z_w = []
        for v in problem.getVars():
            if 'z' in v.varName:
                z_w.append(v.x)
        x_w = z_w[0:x_dim]
        y_w = z_w[x_dim:(x_dim+y_dim)]
        x_all_scenarios.append(x_w)
        y_all_scenarios.append(y_w)
        z_all_scenarios.append(z_w)
        run_time.append(problem.Runtime)
        all_run_time.append(problem.Runtime)
    
    x_list[cur_iter] = x_all_scenarios
    y_list[cur_iter] = y_all_scenarios
    print('----------Iteration %d----------' %cur_iter)
    print('x solution: ' )
    print(x_all_scenarios)
    print('y solution: ' )
    print(y_all_scenarios)
    print('Running time: ')
    print(run_time)
    
    for w in range(scenario_num):
        print('Ojective for scenario %d' %w)
        print(objectives[w])
    
    # PHA: Price update
    for w in range(scenario_num):
        omega_list_copy[w] += p3*(np.asarray(x_all_scenarios[w]) - x_hat)
    # PHA: Iteration update
    cur_iter += 1

## PHA with D-Wave (w QUBO form)

In [ ]:
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

def matrix_to_qubo_helper(A):
    Q = {}
    for i in range(A.shape[1]):
        for j in range(i, A.shape[1]):
            if i != j:
                Q[(i,j)] = A[i][j] + A[j][i]
            else:
                Q[(i,j)] = A[i][j]
    return Q

useQPU = True

In [ ]:
num_iterations = 50
x_list = [0]*num_iterations
y_list = [0]*num_iterations
cur_iter = 0

x_all_scenarios = []
y_all_scenarios = []
objectives = []

p1 = 500
p2 = 500
p3 = 500

# PHA: Initialization 
for w in range(scenario_num):
    ##### Compute QUBO Q matrix ######
    dwave_Q = np.zeros((x_dim + y_dim, x_dim + y_dim))
    dwave_Q[0:x_dim, 0:x_dim] = np.diag(c) + 0.5*Q + p1*np.matmul(np.transpose(A),A) - 2*p1*np.diag(np.matmul(np.transpose(A),b)) \
                                + p2*np.matmul(np.transpose(T_list[w]),T_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(T_list[w]),h_list[w]))
    dwave_Q[x_dim:(x_dim+y_dim), x_dim:(x_dim+y_dim)] = np.diag(q_list[w]) + 0.5*D_list[w] + \
                                + p2*np.matmul(np.transpose(W_list[w]),W_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(W_list[w]),h_list[w]))
    T_m_W = np.matmul(np.transpose(T_list[w]),W_list[w])
    for i in range(x_dim):
        for j in range(y_dim):
            x_index = i
            y_index = j + x_dim
            dwave_Q[x_index, y_index] = p2*T_m_W[i,j]
            dwave_Q[y_index, x_index] = p2*T_m_W[i,j]
    offset = p1*np.matmul(np.transpose(b),b) + p2*np.matmul(np.transpose(h_list[w]),h_list[w])
    ###################################
    bqm = dimod.BinaryQuadraticModel.from_qubo(matrix_to_qubo_helper(dwave_Q), offset = offset)
    if useQPU:
        sampler = EmbeddingComposite(DWaveSampler(profile='junsong@berkeley.edu'))
        sample_set = sampler.sample(bqm, num_reads=3500)
        print(sample_set.info["timing"]) 
    else: 
        sampler = dimod.ExactSolver()
        sample_set = sampler.sample(bqm)

    # Use the lowest energy solution
    x_w = list(sample_set.samples()[0].values())[0:x_dim]
    y_w = list(sample_set.samples()[0].values())[x_dim:(x_dim+y_dim)]
    objectives.append(min(sample_set.record['energy']))
    x_all_scenarios.append(x_w)
    y_all_scenarios.append(y_w)
            
x_list[cur_iter] = x_all_scenarios
y_list[cur_iter]= y_all_scenarios
print('----------Iteration %d----------' %cur_iter)
print('x solution: ' )
print(x_all_scenarios)
print('y solution: ' )
print(y_all_scenarios)
for w in range(scenario_num):
    print('Ojective for scenario %d' %w)
    print(objectives[w])
cur_iter += 1

omega_list_copy = []
for w in range(scenario_num):
    omega_list_copy.append(np.zeros(x_dim)) # initiate price to zero
    
while (not x_all_scenarios.count(x_all_scenarios[0]) == len(x_all_scenarios)) and (cur_iter < num_iterations):
    # PHA: Aggregation
    x_hat = np.zeros(x_dim)
    for w in range(scenario_num):
        x_hat += p_xi_list[w]*np.asarray(x_all_scenarios[w])
    
    # PHA: Decomposition
    x_all_scenarios = []
    y_all_scenarios = []
    objectives = []
    for w in range(scenario_num):
        ##### Compute QUBO Q matrix #######
        dwave_Q = np.zeros((x_dim + y_dim, x_dim + y_dim))
        dwave_Q[0:x_dim, 0:x_dim] = np.diag(c) + 0.5*Q + p1*np.matmul(np.transpose(A),A) - 2*p1*np.diag(np.matmul(np.transpose(A),b)) \
                                + p2*np.matmul(np.transpose(T_list[w]),T_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(T_list[w]),h_list[w])) \
                                + np.diag(omega_list_copy[w]) + 0.5*p3*np.identity(x_dim) - p3*np.diag(x_hat)
        dwave_Q[x_dim:(x_dim+y_dim), x_dim:(x_dim+y_dim)] = np.diag(q_list[w]) + 0.5*D_list[w] + \
                                + p2*np.matmul(np.transpose(W_list[w]),W_list[w]) - 2*p2*np.diag(np.matmul(np.transpose(W_list[w]),h_list[w]))
        T_m_W = np.matmul(np.transpose(T_list[w]),W_list[w])
        for i in range(x_dim):
            for j in range(y_dim):
                x_index = i
                y_index = j + x_dim
                dwave_Q[x_index, y_index] = p2*T_m_W[i,j]
                dwave_Q[y_index, x_index] = p2*T_m_W[i,j]
        offset = p1*np.matmul(np.transpose(b),b) + p2*np.matmul(np.transpose(h_list[w]),h_list[w]) + 0.5*p3*np.matmul(np.transpose(x_hat),x_hat)
        ###################################
        bqm = dimod.BinaryQuadraticModel.from_qubo(matrix_to_qubo_helper(dwave_Q), offset = offset)
        if useQPU:
            sampler = EmbeddingComposite(DWaveSampler(profile='junsong@berkeley.edu'))
            sample_set = sampler.sample(bqm, num_reads=3500)
            print(sample_set.info["timing"]) 
        else: 
            sampler = dimod.ExactSolver()
            sample_set = sampler.sample(bqm)
    
        # Extract the lowest energy solution
        x_w = list(sample_set.samples()[0].values())[0:x_dim]
        y_w = list(sample_set.samples()[0].values())[x_dim:(x_dim+y_dim)]
        objectives.append(min(sample_set.record['energy']))
        x_all_scenarios.append(x_w)
        y_all_scenarios.append(y_w)
    
    x_list[cur_iter] = x_all_scenarios
    y_list[cur_iter] = y_all_scenarios
    print('----------Iteration %d----------' %cur_iter)
    print('x solution: ' )
    print(x_all_scenarios)
    print('y solution: ' )
    print(y_all_scenarios)
    for w in range(scenario_num):
        print('Ojective for scenario %d' %w)
        print(objectives[w])
    # PHA: Price update
    for w in range(scenario_num):
        omega_list_copy[w] += p3*(np.asarray(x_all_scenarios[w]) - x_hat)
    # PHA: Iteration update
    cur_iter += 1